<a href="https://colab.research.google.com/github/hr1588/NLP/blob/main/chapter_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/rickiepark/nlp-with-transformers.git
%cd nlp-with-transformers
from install import *
install_requirements(chapter=6)

Cloning into 'nlp-with-transformers'...
remote: Enumerating objects: 588, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 588 (delta 5), reused 3 (delta 1), pack-reused 569
Receiving objects: 100% (588/588), 57.41 MiB | 20.34 MiB/s, done.
Resolving deltas: 100% (293/293), done.
/content/nlp-with-transformers
⏳ Installing base requirements ...
✅ Base requirements installed!
Using transformers v4.27.3
Using datasets v2.10.1
Using accelerate v0.18.0
Using sentencepiece v0.1.97
Using sacrebleu v2.3.1
Using rouge_score
Using nltk v3.8.1
Using py7zr v0.20.4


In [2]:
from transformers import pipeline, set_seed

# CNN/DailyMail 데이터셋

In [3]:
from datasets import load_dataset

dataset = load_dataset('cnn_dailymail', version = '3.0.0')
print(f"특성: {dataset['train'].column_names}")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/default/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

특성: ['article', 'highlights', 'id']


In [4]:
sample = dataset['train'][1]
print(f"""기사 (500개 문자 발췌, 총 길이 : {len(sample['article'])}):""")
print(sample['article'][:500])
print(f'\n요약 (길이 : {len(sample["highlights"])}):')
print(sample['highlights'])

기사 (500개 문자 발췌, 총 길이 : 4051):
Editor's note: In our Behind the Scenes series, CNN correspondents share their
experiences in covering news and analyze the stories behind the events. Here,
Soledad O'Brien takes users inside a jail where many of the inmates are mentally
ill. An inmate housed on the "forgotten floor," where many mentally ill inmates
are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the
Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here,
inmates with the most s

요약 (길이 : 281):
Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .


# 텍스트 요약 파이프라인

In [5]:
sample_text = dataset['train'][1]['article'][:2000]
summaries = {}

In [6]:
import nltk
from nltk.tokenize import sent_tokenize

nltk.download('punkt')
string = "The U.S. are a country. The U.N. is an organization."
sent_tokenize(string)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


['The U.S. are a country.', 'The U.N. is an organization.']

# 요약 기준 모델

In [7]:
def three_sentence_summary(text):
    return "\n".join(sent_tokenize(text)[:3]) 

In [8]:
summaries['baseline'] = three_sentence_summary(sample_text)

In [9]:
summaries # 단순히 기사에서 맨 처음 문장 3개를 선택

{'baseline': 'Editor\'s note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events.\nHere, Soledad O\'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial.\nMIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor."'}

In [10]:
sample_text

'Editor\'s note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O\'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most severe mental illnesses are incarcerated until they\'re ready to appear in court. Most often, they face drug charges or charges of assaulting an officer --charges that Judge Steven Leifman says are usually "avoidable felonies." He says the arrests often result from confrontations with police. Mentally ill people often won\'t do what they\'re told when police arrive on the scene -- confrontation seems to exacerbate their illness and they become more paranoid, delusional, and less likely to foll

## gpt-2

In [11]:
def pipe_function(model):
    return pipeline("text-generation", model = model)

In [12]:
from transformers import pipeline, set_seed

set_seed(42)

pipe = pipe_function('gpt2')
gpt2_query = sample_text + "\nTL;DR:\n"
pipe_out = pipe(gpt2_query, max_length = 512, clean_up_tokenization_spaces = True)
summaries['gpt2'] = "\n".join(sent_tokenize(pipe_out[0]['generated_text'][len(gpt2_query) :]))

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


- 파이썬에서 "\n"은 줄 바꿈 문자를 나타내는 특수 문자 시퀀스이다. 출력을 콘솔로 인쇄할 때 줄 바꿈을 삽입하거나 새 줄을 만드는 데 사용

- "TL;DR"은 "Too Long; Don't Read"를 의미하며, 일반적으로 더 긴 텍스트나 메시지의 요약으로 사용

- \nTL;파이썬에서 DR:\n"은 줄 바꿈으로 이전의 텍스트와 구분하여 새 줄에 요약이나 간단한 문장을 인쇄하는 데 사용

In [13]:
gpt2_query

'Editor\'s note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O\'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most severe mental illnesses are incarcerated until they\'re ready to appear in court. Most often, they face drug charges or charges of assaulting an officer --charges that Judge Steven Leifman says are usually "avoidable felonies." He says the arrests often result from confrontations with police. Mentally ill people often won\'t do what they\'re told when police arrive on the scene -- confrontation seems to exacerbate their illness and they become more paranoid, delusional, and less likely to foll

In [14]:
summaries

{'baseline': 'Editor\'s note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events.\nHere, Soledad O\'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial.\nMIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor."',
 'gpt2': "\nThe 9th floor has a prison facility.\nIt's basically the home of prisoners charged with crimes because of their mental illnesses.\nThey're locked in a long hallway.\nThat hallway is a narrow hall, and there is a large metal prison cell near the exit to the hallway (to accommodate cellmates to help them escape or get a way out of the cell, for example).\nPrison"}

## T5

In [15]:
def summarize(model, sample_text):
    pipe = pipeline('summarization', model = model)
    pipe_out = pipe(sample_text, max_length = 512, clean_up_tokenization_spaces = True)
    return "\n".join(sent_tokenize(pipe_out[0]['summary_text']))

In [16]:
summaries["t5"] = summarize('t5-small', sample_text)

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


# BART

In [17]:
summaries['bart'] = summarize('facebook/bart-large-cnn', sample_text)

## PEGASUS

In [18]:
summaries['pegasus'] = summarize('google/pegasus-cnn_dailymail', sample_text)

# 요약 결과 비교

In [19]:
print("GROUND TRUTH")
print(dataset["train"][1]["highlights"])
print("")

for model_name in summaries:
    print(model_name.upper())
    print(summaries[model_name])
    print("")

GROUND TRUTH
Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .

BASELINE
Editor's note: In our Behind the Scenes series, CNN correspondents share their
experiences in covering news and analyze the stories behind the events.
Here, Soledad O'Brien takes users inside a jail where many of the inmates are
mentally ill. An inmate housed on the "forgotten floor," where many mentally ill
inmates are housed in Miami before trial.
MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention
facility is dubbed the "forgotten floor."

GPT2

The 9th floor has a prison facility.
It's basically the home of prisoners charged with crimes because of their mental
illnesses.
They're locked in a long hallway.
That hallway is a narrow hall, and there is a large me

# BLEU

- 1부터 N까지 n-gram에서 수정 정밀도의 기하 평균
- 텍스트 평가에 널리 사용되며, 특히 기계 번역에 많이 사용됨
- smooth_method가 floor이면 smooth_value의 기본값이 0.1이고, n-gram이 없을 경우 분자로 0.1을 사용.
- 여기서는 점수가 없으면 0을 만들기 위해 값을 0으로 지정

In [26]:
from datasets import load_metric
import pandas as pd
import numpy as np

bleu_metric = load_metric('sacrebleu')
bleu_metric.add(prediction = "the the the the the the", reference = ["the cat is on the mat"])

results = bleu_metric.compute(smooth_method='floor', smooth_value = 0)
results['precisions'] = [np.round(p,2) for p in results['precisions']]
pd.DataFrame.from_dict(results, orient= 'index', columns = ['Value'])

,Value
score,0.0
counts,"[2, 0, 0, 0]"
totals,"[6, 5, 4, 3]"
precisions,"[33.33, 0.0, 0.0, 0.0]"
bp,1.0
sys_len,6
ref_len,6


In [27]:
results

{'score': 0.0,
 'counts': [2, 0, 0, 0],
 'totals': [6, 5, 4, 3],
 'precisions': [33.33, 0.0, 0.0, 0.0],
 'bp': 1.0,
 'sys_len': 6,
 'ref_len': 6}

In [29]:
bleu_metric.add(prediction = "the cat is on mat", reference = ["the cat is on the mat"])

results = bleu_metric.compute(smooth_method='floor', smooth_value = 0)
results['precisions'] = [np.round(p,2) for p in results['precisions']]
pd.DataFrame.from_dict(results, orient= 'index', columns = ['Value'])

,Value
score,57.893007
counts,"[5, 3, 2, 1]"
totals,"[5, 4, 3, 2]"
precisions,"[100.0, 75.0, 66.67, 50.0]"
bp,0.818731
sys_len,5
ref_len,6


# ROUGE

In [30]:
rouge_metric = load_metric('rouge')

In [32]:
summaries

{'baseline': 'Editor\'s note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events.\nHere, Soledad O\'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial.\nMIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor."',
 'gpt2': "\nThe 9th floor has a prison facility.\nIt's basically the home of prisoners charged with crimes because of their mental illnesses.\nThey're locked in a long hallway.\nThat hallway is a narrow hall, and there is a large metal prison cell near the exit to the hallway (to accommodate cellmates to help them escape or get a way out of the cell, for example).\nPrison",
 't5': "inmates with most severe mental illnesses are incarcerated until they're ready to appear in court.\nmost often, they face dr

In [34]:
reference = dataset["train"][1]['highlights']
records = []
rouge_names = ['rouge1','rouge2','rougeL','rougeLsum']

for model_name in summaries:
    rouge_metric.add(prediction = summaries[model_name], reference = reference)
    score = rouge_metric.compute()
    rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
    records.append(rouge_dict)

pd.DataFrame.from_records(records, index = summaries.keys())

,rouge1,rouge2,rougeL,rougeLsum
baseline,0.365079,0.145161,0.206349,0.285714
gpt2,0.258621,0.017544,0.155172,0.241379
t5,0.195652,0.022222,0.108696,0.173913
bart,0.475248,0.222222,0.316832,0.415842
pegasus,0.316832,0.202020,0.277228,0.316832


- bart model의 rouge값이 제일 높게 나옴을 확인

# dataset에서 BART 평가

In [ ]:
# 노트북 중간부터 실행할 때 사용
import matplotlib.pyplot as plt
import pandas as pd
from datasets import load_dataset, load_metric
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# "cnn_dailymail" 데이터셋 다운로드 에러가 발생할 경우 대신 "ccdv/cnn_dailymail"을 사용하세요.
dataset = load_dataset("cnn_dailymail", version="3.0.0") 
rouge_metric = load_metric("rouge", cache_dir=None)
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

## baseline

In [40]:
def evaluate_summaries_baseline(dataset, metric,
                                column_text = 'article',
                                column_summary = 'highlights'):
    summaries = [three_sentence_summary(text) for text in dataset[column_text]]
    metric.add_batch(predictions = summaries,
                     references = dataset[column_summary])
    score = metric.compute()
    return score

In [41]:
test_sampled = dataset['test'].shuffle(seed=42).select(range(1000)) # 빠른 계산을 위해 테스트 세트에서 1000개 sampling

score = evaluate_summaries_baseline(test_sampled, rouge_metric)
rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame.from_dict(rouge_dict, orient = 'index', columns = ['baseline']).T

,rouge1,rouge2,rougeL,rougeLsum
baseline,0.389276,0.171296,0.245061,0.354239


## BART

In [43]:
from tqdm import tqdm
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

def chunks(list_of_elements, batch_size):
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i:i+batch_size]

def evaluate_summaries_bart(dataset, metric, model, tokenizer, batch_size = 16, device = device,
                            column_text = 'article', column_summary = 'highlights'):
    article_batches = list(chunks(dataset[column_text], batch_size))
    target_batches = list(chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(zip(article_batches, target_batches), total=len(article_batches)):
        inputs = tokenizer(article_batch, max_length = 1024, truncation = True,
                           padding = "max_length", return_tensors = "pt")
        
        summaries = model.generate(input_ids = inputs['input_ids'].to(device),
                                   attention_mask = inputs['attention_mask'].to(device),
                                   length_penalty = 0.8, num_beams = 8, max_length = 128
                                   )
        
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens = True, clean_up_tokenization_spaces = True) for s in summaries]
        decoded_summaries = [d.replace("<n>"," ") for d in decoded_summaries]
        metric.add_batch(predictions = decoded_summaries, references = target_batch)
    
    score = metric.compute()
    return score

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_ckpt = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)
score = evaluate_summaries_bart(test_sampled, rouge_metric,
                                model, tokenizer, batch_size = 8)

rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame(rouge_dict, index = ["bart"])

 36%|███▌      | 45/125 [09:17<17:42, 13.29s/it]

# 요약 모델 훈련하기

In [ ]:
dataset_samsum = load_dataset("samsum")
split_lengths = [len(dataset_samsum[split])for split in dataset_samsum]

print(f"분할 크기 :{split_lengths}")
print(f"특성: {dataset_samsum['train'].column_names}")
print("\n대화:")
print(dataset_samsum['test'][0]['dialogue'])
print('\nSummary:')
print(dataset_samsum['test'][0]['summary'])

## samsum에서 bart 평가

In [ ]:
pipe_out = pipe(dataset_samsum['test'][0]['dialogue'])
print("요약:")
print(pipe_out[0]['summary_text']).replace(" .<n>","\n")

score = evaluate_summaries_bart(dataset_samsum['test'], rouge_metric, model,
                                tokenizer, column_text = 'dialogue',
                                column_summary = 'summary', batch_size = 8)

rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rough_names)
pd.DataFrame(rouge_dict, index = ['bart'])

## bart fine-tuning

In [1]:
d_len = [len(tokenizer.encode(s)) for s in dataset_samsum['train']['dialogue']]
s_len = [len(tokenizer.encode(s)) for s in dataset_samsum['train']['summary']]

fig, axes = plt.subplots(1,2, figsize = (10,3,5), sharey = True) # sharey : y축범위 통일
axes[0].hist(d_len, bins=20, color="C0", edgecolor="C0")
axes[0].set_title("Dialogue Token Length")
axes[0].set_xlabel("Length")
axes[0].set_ylabel("Count")
axes[1].hist(s_len, bins=20, color="C0", edgecolor="C0")
axes[1].set_title("Summary Token Length")
axes[1].set_xlabel("Length")
plt.tight_layout()
plt.show()

NameError: ignored

### trainer를 위한 data-collator

In [ ]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['dialogue'], max_length = 1024, truncation = True)

    with tokenizer.as_target_tokenizer(): # 인코더와 디코더 입력의 토큰화 구분을 위해 사용
        target_encodings = tokenizer(example_batch['summary'], max_length = 128, truncation = True)

        return {"input_ids": input_encodings['input_ids'],
                "attention_mask": input_encodings["attention_mask"],
                "labels": target_encodings["input_ids"]}

dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features, batched = True)
columns = ['input_ids','labels','attention_mask']

In [ ]:
# 티처 포싱(teacher forcing) : 텍스트 입력을 위한 디코더 입력과 label 정렬

text = ['PAD','Transformers', 'are', 'awesome', 'for', 'text', 'summarization']
rows = []
for i in range(len(text)-1):
    rows.append({'step': i+1, 'decoder_input': text[:i+1], 'label': text[i+1]})
pd.DataFrame(rows).set_index('step')

In [ ]:
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='pegasus-bart', num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, logging_steps=10, push_to_hub=True,
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16) # 작은 배치를 만들고 gradient를 누적하는 방식

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
trainer = Trainer(model=model, args=training_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["train"], 
                  eval_dataset=dataset_samsum_pt["validation"])

print(trainer.train())
score = evaluate_summaries_bart(
    dataset_samsum["test"], rouge_metric, trainer.model, tokenizer,
    batch_size=2, column_text="dialogue", column_summary="summary")

rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
print(pd.DataFrame(rouge_dict, index=[f"bart"]))

In [ ]:
trainer.push_to_hub("Training complete!")